In [33]:
import cv2
import numpy as np
import handLandmarkMP as hlm
import time
import math
from blunet import connect, send

In [34]:
hands = hlm.HandDetector()

In [35]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = int(Dx / D)
        y = int(Dy / D)
        return x,y
    else:
        return False

In [ ]:
connection = connect("HC-05", "1234")

In [59]:
cap = cv2.VideoCapture(0)
success, img = cap.read()
ref_p1 = np.array((0, img.shape[0]-30))
ref_p2 = np.array((img.shape[1], img.shape[0]-30))
intersection_point = (ref_p1 + ref_p2) // 2
R_before = 0
R = 0
angle_before = 90
while cap.isOpened():
    success, img = cap.read()
    img = cv2.flip(img, 1)
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue
    connection.send(str(angle_before // 2))
    img = hands.findHands(img)
    hands_positions = hands.findPositions(img)
    
    if hands_positions:
        #middle = np.array(hands_positions['hand1']['middle'][0])
        
        wrist = np.array(hands_positions['hand1']['wrist'])
        img = cv2.circle(img, wrist, 0, (255, 255, 255), 10)
        
        
        #img = cv2.circle(img, middle, 0, (255, 255, 255), 10)
        #img = cv2.line(img, middle, wrist, (0, 0, 0), 2)
        vertical_dist = intersection_point[1] - wrist[1]
        horizontal_dist = intersection_point[0] - wrist[0]
        hyp = int(np.sqrt((vertical_dist)**2 + (horizontal_dist)**2))
        #print(f"hyp = {hyp}")
        #print(f"vertical_dist = {vertical_dist}")
        #print(f"wrist = {wrist}")
        
        angle = int(math.degrees(np.arccos(horizontal_dist / hyp)))
        if angle not in range(angle_before-10,angle_before+10):
            print(f"angle sent {str(angle // (2 * 10))}")
            connection.send(str(angle // 2))
            angle_before = angle
        
        img = cv2.line(img, ref_p1, ref_p2, (0, 0, 0), 2)
        img = cv2.circle(img, intersection_point, 0, (0, 0, 255), 10)
        
    
    #L1 = line(middle, wrist)
    #L2 = line(ref_p1, ref_p2)
    #R = intersection(L1, L2)
    #if(np.all(np.isclose(R,R_before,atol = 30))):
    #R = R_before
    
    img = cv2.line(img, intersection_point, wrist, (255, 255, 255), 2)
    R_before = R
    cv2.imshow("Image", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

angle sent 5
angle sent 5
angle sent 6
angle sent 7
angle sent 7
angle sent 8
angle sent 8
angle sent 7
angle sent 7
angle sent 6
angle sent 5
angle sent 5
angle sent 4
angle sent 4
angle sent 3
angle sent 2
angle sent 2
angle sent 1
angle sent 1
angle sent 1
angle sent 2
angle sent 3
angle sent 4
angle sent 5
angle sent 6
angle sent 7
angle sent 7
angle sent 8


In [20]:
connection = connect("HC-05", "1234")

Bluetooth module found
Connected Successfully
